https://github.com/richliao/textClassifier/blob/master/textClassifierRNN.py

In [1]:
import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re

In [2]:
from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='theano'

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using Theano backend.
/home/root1/.virtualenv/demos/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [5]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [6]:
os.chdir("../../")
cwd = os.getcwd()
print cwd



DATA_FOLDER = os.path.join(cwd, 'Data')
print DATA_FOLDER

/home/root1/anuj_work/Github_repos/DeepNets
/home/root1/anuj_work/Github_repos/DeepNets/Data


In [7]:
file_path = os.path.join(DATA_FOLDER, 'imdb_labeledTrainData.tsv')

In [8]:
data_train = pd.read_csv(file_path, sep='\t')

In [9]:
print data_train.shape

(25000, 3)


In [11]:
data_train.head(5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [12]:
texts = []
labels = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx])
    texts.append(clean_str(text.get_text().encode('ascii','ignore')))
    labels.append(data_train.sentiment[idx])

/home/root1/.virtualenv/demos/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [19]:
texts[:1]

['with all this stuff going down at the moment with mj ive started listening to his music, watching the odd documentary here and there, watched the wiz and watched moonwalker again. maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. some of it has subtle messages about mjs feeling towards the press and also the obvious message of drugs are bad mkay.visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring. some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him.the actual feature film bit when it finally starts is only on for 20 minu

In [20]:
labels[:1]

[1]

In [17]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


/home/root1/.virtualenv/demos/local/lib/python2.7/site-packages/keras/preprocessing/text.py:89: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [21]:
sequences[:1]

[[15,
  29,
  10,
  536,
  165,
  177,
  30,
  1,
  559,
  15,
  10233,
  198,
  645,
  2628,
  5,
  23,
  223,
  144,
  1,
  1024,
  656,
  128,
  2,
  46,
  291,
  1,
  19971,
  2,
  291,
  11752,
  169,
  275,
  9,
  40,
  180,
  5,
  75,
  3,
  807,
  2621,
  80,
  10,
  226,
  34,
  9,
  193,
  12,
  62,
  638,
  7,
  1,
  4269,
  40,
  5,
  275,
  93,
  52,
  57,
  325,
  723,
  26,
  6,
  2519,
  39,
  1349,
  11752,
  6,
  168,
  5059,
  168,
  779,
  18,
  59,
  9,
  373,
  165,
  5,
  63,
  30,
  1,
  432,
  50,
  8,
  12,
  1820,
  624,
  45,
  4,
  8,
  44,
  1297,
  3454,
  41,
  544,
  944,
  1,
  3536,
  2,
  78,
  1,
  577,
  745,
  4,
  1658,
  22,
  73,
  2013,
  1154,
  17,
  4,
  259,
  10,
  6,
  29,
  41,
  485,
  1877,
  35,
  894,
  21,
  2596,
  37,
  10233,
  7,
  553,
  90,
  21,
  22,
  165,
  5,
  780,
  10,
  2,
  164,
  8,
  355,
  45,
  199,
  678,
  10233,
  32,
  14,
  5,
  1,
  227,
  4,
  10,
  16,
  17,
  10233,
  2,
  87,
  4,
  23,
  444,
  58,
  

In [22]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 80568 unique tokens.


In [23]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (25000, 1000))
('Shape of label tensor:', (25000, 2))


In [24]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [25]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Traing and validation set number of positive and negative reviews')
print y_train.sum(axis=0)
print y_val.sum(axis=0)

Traing and validation set number of positive and negative reviews
[ 9998. 10002.]
[2502. 2498.]


In [26]:
cwd = os.getcwd()
print cwd

/home/root1/anuj_work/Github_repos/DeepNets


In [28]:
GLOVE_FOLDER = os.path.join(cwd, 'Data/glove')
print GLOVE_FOLDER

#GLOVE_DIR = "~/Data/glove"

embeddings_index = {}
f = open(os.path.join(GLOVE_FOLDER, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

/home/root1/anuj_work/Github_repos/DeepNets/Data/glove


In [29]:
print('Total %s word vectors.' % len(embeddings_index))


Total 400000 word vectors.


In [30]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [31]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [32]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)
model = Model(sequence_input, preds)

In [33]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
print("model fitting - Bidirectional LSTM")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epoch=10, batch_size=50)

model fitting - Bidirectional LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8056900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,218,102
Trainable params: 8,218,102
Non-trainable params: 0
_________________________________________________________________


/home/root1/.virtualenv/demos/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
   50/20000 [..............................] - ETA: 23936s - loss: 0.7467 - acc: 0.4400